# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [7]:
#Your code here
import requests
import time
import pandas as pd

start = time.time()

client_id = 'bVX1Jsfp4dkIOqw5HOVplg' #Your client ID goes here (as a string)
api_key = 'RTzp-q-TgkJW_NFQogubFvZNRDziXyoR38VbtZMWibDI-FlvB25OE7GmafFEqhTL8_Bk2HlcX24-hRWLMP7Nc6WHO_VXMXldpPBjP0LoPv5EFFELMSI2oll8njhbXHYx' #Your api key goes here (as a string)

term = 'Vegetarian'
location = 'Elmhurst NY'
search_limit = 50
offset_amount = 0

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': search_limit,
                'offset': offset_amount
            }

# response = requests.get(url, headers=headers, params=url_params)
# yelp_data = response.json()
full_yelp_data = []
max_results = 1000

# for business in yelp_data['businesses']:  #First 50 results
#     full_yelp_data.append(business)


# Your code here; use a function or loop to retrieve all the results from your original request

while (offset_amount - 1) < max_results:
    response = requests.get(url, headers=headers, params=url_params)
    yelp_data = response.json()
    max_results = min(yelp_data['total'], 1000)
    for business in yelp_data['businesses']:
        full_yelp_data.append(business)
    offset_amount = len(full_yelp_data) + 1
    time.sleep(.5)

end = time.time()

print(end - start)
print(len(full_yelp_data))
df = pd.DataFrame(full_yelp_data)
print(df.head())
# print(response.json().keys())
# response.json()['total'] #1200
# dict(response.headers)



29.65499448776245
1000
                                               alias  \
0                         green-zenphony-rego-park-2   
1  samudra-vegetarian-restaurant-and-chaat-house-...   
2                                      joju-elmhurst   
3                          tawa-food-jackson-heights   
4                                jujube-tree-astoria   

                                          categories  \
0  [{'alias': 'vegetarian', 'title': 'Vegetarian'...   
1  [{'alias': 'indpak', 'title': 'Indian'}, {'ali...   
2  [{'alias': 'vietnamese', 'title': 'Vietnamese'...   
3  [{'alias': 'himalayan', 'title': 'Himalayan/Ne...   
4  [{'alias': 'vegan', 'title': 'Vegan'}, {'alias...   

                                         coordinates   display_phone  \
0   {'latitude': 40.730219, 'longitude': -73.863865}  (718) 830-0808   
1  {'latitude': 40.7490953, 'longitude': -73.8903...  (718) 255-1757   
2  {'latitude': 40.7407051700423, 'longitude': -7...  (347) 808-0887   
3  {'latitude':

In [8]:
data = pd.DataFrame(df)
data.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,green-zenphony-rego-park-2,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...","{'latitude': 40.730219, 'longitude': -73.863865}",(718) 830-0808,1716.297530,DPOWubhvyzYGjLbVvbr_sg,https://s3-media2.fl.yelpcdn.com/bphoto/_2JZ2t...,False,"{'address1': '95-26 Queens Blvd', 'address2': ...",Green Zenphony,+17188300808,$$,4.5,171,"[delivery, pickup]",https://www.yelp.com/biz/green-zenphony-rego-p...
1,samudra-vegetarian-restaurant-and-chaat-house-...,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...","{'latitude': 40.7490953, 'longitude': -73.8903...",(718) 255-1757,1435.576809,XUataECnaOfdo4a87NqEFg,https://s3-media1.fl.yelpcdn.com/bphoto/fv7yXn...,False,"{'address1': '75-18 37th Ave', 'address2': '',...",Samudra Vegetarian Restaurant & Chaat House,+17182551757,$$,4.0,484,"[delivery, pickup]",https://www.yelp.com/biz/samudra-vegetarian-re...
2,joju-elmhurst,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...","{'latitude': 40.7407051700423, 'longitude': -7...",(347) 808-0887,329.896360,JlFbaMbMzsu7FA307mUIqg,https://s3-media3.fl.yelpcdn.com/bphoto/4quqnl...,False,"{'address1': '83-25 Broadway', 'address2': '',...",JoJu,+13478080887,$,4.5,1491,[pickup],https://www.yelp.com/biz/joju-elmhurst?adjust_...
3,tawa-food-jackson-heights,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...","{'latitude': 40.7476070420478, 'longitude': -7...",(718) 457-7766,1496.329321,LaMbEoM5Vw9CQP5usaD2xA,https://s3-media2.fl.yelpcdn.com/bphoto/3r2ffG...,False,"{'address1': '3738 72nd St', 'address2': '', '...",Tawa Food,+17184577766,$,4.0,86,[pickup],https://www.yelp.com/biz/tawa-food-jackson-hei...
4,jujube-tree-astoria,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...","{'latitude': 40.76512, 'longitude': -73.91821}",(718) 545-1888,4317.517354,S7ivv5O0beH3U9cV3cRD_Q,https://s3-media2.fl.yelpcdn.com/bphoto/UMwvZa...,False,"{'address1': '35-02 30th Ave', 'address2': '',...",Jujube Tree,+17185451888,$$,4.5,341,"[delivery, pickup]",https://www.yelp.com/biz/jujube-tree-astoria?a...


In [3]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print(value)
    print('\n\n') #Seperate out data
    
# yelp_data['name'].unique()
# print(yelp_data['businesses'])
for biz in yelp_data['businesses']:
    print(biz)
    print('\n\n')

businesses
<class 'list'>
[{'id': 'DPOWubhvyzYGjLbVvbr_sg', 'alias': 'green-zenphony-rego-park-2', 'name': 'Green Zenphony', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/_2JZ2t2PTF5jPEkIKY1I-g/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/green-zenphony-rego-park-2?adjust_creative=bVX1Jsfp4dkIOqw5HOVplg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=bVX1Jsfp4dkIOqw5HOVplg', 'review_count': 171, 'categories': [{'alias': 'vegetarian', 'title': 'Vegetarian'}, {'alias': 'chinese', 'title': 'Chinese'}, {'alias': 'vegan', 'title': 'Vegan'}], 'rating': 4.5, 'coordinates': {'latitude': 40.730219, 'longitude': -73.863865}, 'transactions': ['pickup', 'delivery'], 'price': '$$', 'location': {'address1': '95-26 Queens Blvd', 'address2': '', 'address3': '', 'city': 'Rego Park', 'zip_code': '11374', 'country': 'US', 'state': 'NY', 'display_address': ['95-26 Queens Blvd', 'Rego Park, NY 11374']}, 'phone': '+17188300808', 'display_phone': '(718) 830-0808'

## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

#Your code here
data.describe(include = 'all')
# data.head(5)
# data.tail(5)

TypeError: unhashable type: 'list'

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
alias            1000 non-null object
categories       1000 non-null object
coordinates      1000 non-null object
display_phone    1000 non-null object
distance         1000 non-null float64
id               1000 non-null object
image_url        1000 non-null object
is_closed        1000 non-null bool
location         1000 non-null object
name             1000 non-null object
phone            1000 non-null object
price            940 non-null object
rating           1000 non-null float64
review_count     1000 non-null int64
transactions     1000 non-null object
url              1000 non-null object
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 118.2+ KB


In [5]:
data.tail(5)

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
995,sushi-island-elmhurst,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...","{'latitude': 40.7349699, 'longitude': -73.87586}",(718) 803-3033,572.987065,etIbdlTKz2yLkvr9MWNPvw,https://s3-media1.fl.yelpcdn.com/bphoto/JlbQEI...,False,"{'address1': '87-18 Queens Blvd', 'address2': ...",Sushi Island,+17188033033,$$,3.5,321,"[delivery, pickup]",https://www.yelp.com/biz/sushi-island-elmhurst...
996,shanghai-zhen-gong-fu-上海真功夫-elmhurst,"[{'alias': 'shanghainese', 'title': 'Shanghain...","{'latitude': 40.73599, 'longitude': -73.87735}",(718) 672-2200,403.465318,8td6qiBvZ-En6gSpPjXWzQ,https://s3-media1.fl.yelpcdn.com/bphoto/lxIIQN...,False,"{'address1': '86-16 Queens Blvd', 'address2': ...",Shanghai Zhen Gong Fu 上海真功夫,+17186722200,$$,4.0,99,"[delivery, pickup]",https://www.yelp.com/biz/shanghai-zhen-gong-fu...
997,josefinas-restaurant-elmhurst,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...","{'latitude': 40.734255, 'longitude': -73.882604}",(718) 606-9977,446.002326,0MqKMvDvmw6s3mnycYLcog,https://s3-media4.fl.yelpcdn.com/bphoto/mqLJbW...,False,"{'address1': '8440 Grand Ave', 'address2': '',...",Josefina's Restaurant,+17186069977,$,4.5,18,"[delivery, pickup]",https://www.yelp.com/biz/josefinas-restaurant-...
998,sripraphai-thai-restaurant-woodside,"[{'alias': 'thai', 'title': 'Thai'}]","{'latitude': 40.7464987, 'longitude': -73.8992...",(718) 899-9599,1770.338373,0L03onqNwxWLrbb_FCK79A,https://s3-media1.fl.yelpcdn.com/bphoto/DBYmdu...,False,"{'address1': '64-13 39th Ave', 'address2': '',...",SriPraPhai Thai Restaurant,+17188999599,$$,4.0,2235,[],https://www.yelp.com/biz/sripraphai-thai-resta...
999,ok-indo-kirula-enterprise-new-york,"[{'alias': 'grocery', 'title': 'Grocery'}]","{'latitude': 40.7367128, 'longitude': -73.8746...",,585.070021,JbCXC4Btv4CRHYzTlM4Vtw,https://s3-media2.fl.yelpcdn.com/bphoto/jOf3Mq...,False,"{'address1': '8815 Justice Ave', 'address2': '...",OK Indo - Kirula Enterprise,,$,4.0,8,[],https://www.yelp.com/biz/ok-indo-kirula-enterp...


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here
# Hint create new columns for latitude, 
# longitude to use for plotting

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!